# Purpose
to parse the text files from the new RFID system around the hut in wedge 4

# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os, glob, csv, sys

# User defined locations

In [4]:
listoffiles=[]
fld='/Users/dennise/Desktop/SF2022/LOGGER'
for folder in os.listdir(fld):
    if len(folder) > 2:
        folder_path = os.path.join(fld,folder)
        for file_name in os.listdir(folder_path):
            listoffiles.append(os.path.join(folder_path,file_name))
listoffiles[0:10]

['/Users/dennise/Desktop/SF2022/LOGGER\\2022_06\\2022_06_16.log',
 '/Users/dennise/Desktop/SF2022/LOGGER\\2022_06\\2022_06_17.log',
 '/Users/dennise/Desktop/SF2022/LOGGER\\2022_06\\2022_06_18.log',
 '/Users/dennise/Desktop/SF2022/LOGGER\\2022_06\\2022_06_19.log',
 '/Users/dennise/Desktop/SF2022/LOGGER\\2022_06\\2022_06_20.log',
 '/Users/dennise/Desktop/SF2022/LOGGER\\2022_06\\2022_06_21.log',
 '/Users/dennise/Desktop/SF2022/LOGGER\\2022_06\\2022_06_22.log',
 '/Users/dennise/Desktop/SF2022/LOGGER\\2022_06\\2022_06_23.log',
 '/Users/dennise/Desktop/SF2022/LOGGER\\2022_06\\2022_06_24.log',
 '/Users/dennise/Desktop/SF2022/LOGGER\\2022_06\\2022_06_25.log']

In [5]:
txtfile=listoffiles[0]
txtfile

'/Users/dennise/Desktop/SF2022/LOGGER\\2022_06\\2022_06_16.log'

# Functions

In [6]:
def get_txtfile_contents(txtfile):
    dates=[]
    first=[]
    second=[]
    third=[]
    tagvals=[]
    mvs=[]

    # import file
    f = open(txtfile, 'r') 
    for x in f.readlines():
        if "TAG" in x:
            values=[xval.split(' ') for xval in x.split(':')]
            dates.append(values[1][2])
            first.append(values[1][3])
            second.append(values[2][0])
            third.append(values[3][0])
            tagvals.append(values[3][1])
            mvs.append(values[3][2])

    # want this to be a dataframe
    df=pd.DataFrame({'dates':dates,'hour':first,'minute':second,'second':third,'tagvals':tagvals,'mvs':mvs})
    df['converted_tags'] = df.tagvals
    for i in np.arange(0,len(df.tagvals)):
        val=df.converted_tags[i]
        df.converted_tags[i]=int(val[4:],base=16)
    return df

# Exploration

In [11]:
has_run=1
if has_run == 0:
    x=-1
    for txtfile in listoffiles:
        x+=1
        if x==0:
            df_alldata = get_txtfile_contents(txtfile)
        else:
            df = get_txtfile_contents(txtfile)
            df_alldata = pd.concat([df_alldata,df],ignore_index=True)
        if x%10==0:
            print(x)
    df_alldata.to_csv('../data/2022_RFID_hut_data.csv')
else:
    df_alldata = pd.read_csv('/Users/dennise/Desktop/SF2022/2022_RFID_hut_data.csv',index_col=0)

In [12]:
df_alldata

,dates,hour,minute,second,tagvals,mvs,converted_tags
0,06/16/2022,17,33,8.09,3E7.0000001D01,714,7425
1,06/16/2022,17,42,33.58,384.0A0301EAAF,498,43000130223
2,06/16/2022,17,42,33.61,384.0A0301EAAF,512,43000130223
3,06/16/2022,17,42,33.65,384.0A0301EAAF,502,43000130223
4,06/16/2022,17,42,33.68,384.0A0301EAAF,494,43000130223
...,...,...,...,...,...,...,...
87267,07/27/2022,6,24,41.13,3E7.0000001D01,739,7425
87268,07/27/2022,6,34,41.41,3E7.0000001D01,736,7425
87269,07/27/2022,6,44,41.20,3E7.0000001D01,734,7425
87270,07/27/2022,7,4,41.13,3E7.0000001D01,735,7425


In [13]:
unique_tags = np.unique(df_alldata.converted_tags)
unique_tags[1:]

array([43000130206, 43000130217, 43000130223, 43000130232, 43000130236,
       43000130250, 43000130258, 43000130280, 43000130282, 43000130291,
       43000130324, 43000130337, 43000130345, 43000130355, 43000130357,
       43000130359, 43000130399, 67000063703], dtype=int64)

In [14]:
# find the last date the animal was seen
for animal in unique_tags:
    subdf=df_alldata[df_alldata.converted_tags==animal]
    print(animal)
    print(list(subdf.dates)[-1])
    print('   ')

7425
07/27/2022
   
43000130206
06/17/2022
   
43000130217
06/16/2022
   
43000130223
06/26/2022
   
43000130232
07/26/2022
   
43000130236
06/17/2022
   
43000130250
07/26/2022
   
43000130258
06/16/2022
   
43000130280
06/22/2022
   
43000130282
06/17/2022
   
43000130291
07/27/2022
   
43000130324
07/26/2022
   
43000130337
07/26/2022
   
43000130345
07/26/2022
   
43000130355
07/26/2022
   
43000130357
06/16/2022
   
43000130359
06/16/2022
   
43000130399
07/26/2022
   
67000063703
07/26/2022
   


In [28]:
df_alldata = df_alldata[df_alldata.converted_tags != 7425] #removes test from system
df_alldata=df_alldata.reset_index().drop(columns='level_0')
df_alldata

,index,dates,hour,minute,second,tagvals,mvs,converted_tags
0,1,06/16/2022,17,42,33.58,384.0A0301EAAF,498,43000130223
1,2,06/16/2022,17,42,33.61,384.0A0301EAAF,512,43000130223
2,3,06/16/2022,17,42,33.65,384.0A0301EAAF,502,43000130223
3,4,06/16/2022,17,42,33.68,384.0A0301EAAF,494,43000130223
4,5,06/16/2022,17,42,33.71,384.0A0301EAAF,489,43000130223
...,...,...,...,...,...,...,...,...
82839,87255,07/27/2022,4,29,23.43,384.0A0301EAF3,666,43000130291
82840,87256,07/27/2022,4,29,23.46,384.0A0301EAF3,654,43000130291
82841,87257,07/27/2022,4,29,23.49,384.0A0301EAF3,667,43000130291
82842,87258,07/27/2022,4,29,23.52,384.0A0301EAF3,710,43000130291


# TODO next
- make this have relative hours. This should help with later aligning events around midnight
- get timestamps from hut videos, make into csv
- align nearest timestamps, assuming some are lost on either side
- calculate concordance



- add Alec RFID from feeder
- get feeder cam timestamps
- align nearest timestamps, assuming some are lost on either side
- calculate concordance